In [45]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Lectura de datos
Los datos de libros empleados en este notebook han sido extraídos de [Goodreads](https://www.goodreads.com/) mediante webscraping con una versión modificada del código de `goodreads_scraper` de Maria Antoniak y Melanie Walsh disponible de forma pública en: https://github.com/maria-antoniak/goodreads-scraper

Las variables más relevantes de los dataframes de libros con los que vamos a tratar son:
* `title`: Título del libro.
* `series_name`: Nombre de la saga/serie.
* `series_n`: Orden del libro dentro de la saga/serie.
* `num_ratings`: Número total de valoraciones del libro en Goodreadas.
* `average_rating`: Valoración media del libro en Goodreads. El rango es 1-5.
* `hist_rating`: Valoración media ponderada de la saga/libro a la que pertenece. Se calcula haciendo una media ponderada (por el num. de valoraciones) de las puntuaciones del libro en cuestión y los anteriores. Por ej. para calcular `hist_rating` del 3er libro de una saga de 7, se tiene en cuenta la puntuación media de los libros 1, 2 y 3, ponderada por el número de personas que han valorado cada uno de ellos. Esta variable sólo está en el dataframe `sanderson_df`.

In [46]:
sagas_df = pd.read_csv('sagas_df.csv')
sagas_df.head()

,book_id_long,book_id_short,url,title,series_name,series_url,series_n,isbn,year_first_published,author_name,...,num_pages,num_ratings,num_reviews,average_rating,main_genre,rating_5_starts,rating_4_starts,rating_3_starts,rating_2_starts,rating_1_start
0,3.Harry_Potter_and_the_Sorcerer_s_Stone,3,https://www.goodreads.com/book/show/3.Harry_Po...,Harry Potter and the Sorcerer's Stone,Harry Potter,https://www.goodreads.com/series/45175-harry-p...,1,NaN,2003.0,J.K. Rowling,...,309.0,8491079,134158,4.48,Fantasy,5521588,1942578,716350,166014,144549
1,15881.Harry_Potter_and_the_Chamber_of_Secrets,15881,https://www.goodreads.com/book/show/15881.Harr...,Harry Potter and the Chamber of Secrets,Harry Potter,https://www.goodreads.com/series/45175-harry-p...,2,NaN,1999.0,J.K. Rowling,...,341.0,3277548,64524,4.43,Fantasy,1944502,895477,357290,61943,18336
2,5.Harry_Potter_and_the_Prisoner_of_Azkaban,5,https://www.goodreads.com/book/show/5.Harry_Po...,Harry Potter and the Prisoner of Azkaban,Harry Potter,https://www.goodreads.com/series/45175-harry-p...,3,NaN,2004.0,J.K. Rowling,...,435.0,3439727,67846,4.58,Fantasy,2348800,799878,240378,33142,17529
3,6.Harry_Potter_and_the_Goblet_of_Fire,6,https://www.goodreads.com/book/show/6.Harry_Po...,Harry Potter and the Goblet of Fire,Harry Potter,https://www.goodreads.com/series/45175-harry-p...,4,NaN,2002.0,J.K. Rowling,...,734.0,3057818,56382,4.57,Fantasy,2051743,744947,215597,30880,14651
4,2.Harry_Potter_and_the_Order_of_the_Phoenix,2,https://www.goodreads.com/book/show/2.Harry_Po...,Harry Potter and the Order of the Phoenix,Harry Potter,https://www.goodreads.com/series/45175-harry-p...,5,NaN,2004.0,J.K. Rowling,...,870.0,2927014,51572,4.50,Fantasy,1871762,737098,254621,45446,18087


In [47]:
standalone_df  = pd.read_csv('standalone_df.csv')
standalone_df.head()

,book_id_long,book_id_short,url,title,series_name,series_url,series_n,isbn,year_first_published,author_name,...,num_pages,num_ratings,num_reviews,average_rating,main_genre,rating_5_starts,rating_4_starts,rating_3_starts,rating_2_starts,rating_1_start
0,52578297-the-midnight-library,52578297,https://www.goodreads.com/book/show/52578297-t...,The Midnight Library,NaN,NaN,NaN,NaN,2020,Matt Haig,...,304,944368,110251,4.05,Fiction,348885,360185,182361,43080,9857
1,50623864-the-invisible-life-of-addie-larue,50623864,https://www.goodreads.com/book/show/50623864-t...,The Invisible Life of Addie LaRue,NaN,NaN,NaN,NaN,2020,V.E. Schwab,...,444,584015,83315,4.23,Fantasy,282585,190253,82117,21934,7126
2,40961427-1984,40961427,https://www.goodreads.com/book/show/40961427-1984,1984,NaN,NaN,NaN,NaN,2013,George Orwell,...,298,3808026,88601,4.19,Classics,1800853,1238773,543193,146586,78621
3,13079982-fahrenheit-451,13079982,https://www.goodreads.com/book/show/13079982-f...,Fahrenheit 451,NaN,NaN,NaN,B0064CPN7I,2011,Ray Bradbury,...,194,2014586,58859,3.98,Classics,731991,719543,401477,112539,49036
4,5129.Brave_New_World,5129,https://www.goodreads.com/book/show/5129.Brave...,Brave New World,NaN,NaN,NaN,NaN,1998,Aldous Huxley,...,268,1636073,37706,3.99,Classics,600618,586108,321959,88802,38586


In [48]:

books_df_scatter = pd.read_csv('books_df_scatter.csv')
books_df_scatter.head()

,title,series_name,series_n,year_first_published,author_name,num_pages,num_ratings,num_reviews,average_rating,main_genre,saga
0,Harry Potter and the Sorcerer's Stone,Harry Potter,1.0,2003.0,J.K. Rowling,309.0,8491079,134158,4.48,Fantasy,Saga (first book)
1,Harry Potter and the Chamber of Secrets,Harry Potter,2.0,1999.0,J.K. Rowling,341.0,3277548,64524,4.43,Fantasy,Saga (continuation)
2,Harry Potter and the Prisoner of Azkaban,Harry Potter,3.0,2004.0,J.K. Rowling,435.0,3439727,67846,4.58,Fantasy,Saga (continuation)
3,Harry Potter and the Goblet of Fire,Harry Potter,4.0,2002.0,J.K. Rowling,734.0,3057818,56382,4.57,Fantasy,Saga (continuation)
4,Harry Potter and the Order of the Phoenix,Harry Potter,5.0,2004.0,J.K. Rowling,870.0,2927014,51572,4.50,Fantasy,Saga (continuation)


# Gráficos

### Ejemplo 1: Estructura  básica. Scatterplot + cambiar aspecto + añadir líneas y anotaciones

In [49]:
fig_scatter_simple = px.scatter(books_df_scatter, x="num_ratings", y="average_rating", height=500, width=800)
fig_scatter_simple.show()

In [50]:
print(type(fig_scatter_simple))
print(fig_scatter_simple)


<class 'plotly.graph_objs._figure.Figure'>
Figure({
    'data': [{'hovertemplate': 'num_ratings=%{x}<br>average_rating=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([8491079, 3277548, 3439727, ...,   15518,  285067,   16896]),
              'xaxis': 'x',
              'y': array([4.48, 4.43, 4.58, ..., 3.64, 4.46, 3.86]),
              'yaxis': 'y'}],
    'layout': {'height': 500,
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'width': 800,
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'num_ratings'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'average_rating'}}}
})


El tipo de objeto del gráfico creado es `plotly.graph_objs._figure.Figure`, pero a nivel práctico es muy similar a un diccionario (dict) con dos key-value pairs: 
* `data`: información de los datos como tal (datos cuantitativo -ej. valor en el eje x- y cualitativos -ej. grupo/color)
* `layout`:  información del aspecto: colores/paleta, texto, límites, etc

`hovertemplate` está en data en vez de en layout (podría considerarse una parte del aspecto) porque puede incluir otros datos del dataframe que no aparezcan en el gráfico como tal.

La estructura de `go.Figure` es similar: diccionarios anidados.
* Los valores de `data` son LISTAS de diccionarios. En Data hay tantos diccionarios como 'traces' (conjuntos de datos)
* El valor de `layout` es un único diccionario.

Más información: https://plotly.com/python/creating-and-updating-figures/


In [51]:
fig_scatter = px.scatter(
    books_df_scatter, x="num_ratings", y="average_rating", color="saga", 
    hover_data=["title", "series_name", "series_n"],
    height=500, width=800
    )
fig_scatter.show()

In [52]:
# len = 3: cada color es un 'trace'
print(len(fig_scatter['data']))


3


En la celda inferior se usa `pio` para acceder a los templates y crear uno nuevo. Los templates son plantillas de settings del layout (ej. color de fondo, bordes, paleta de colores para cada trace). Hay una serie de templates predeterminados que podemos usar (ej. `'plotly_white'`), pero también podemos crear los nuestros propios con `go.layout.Template`. En el template `borders` lo que especificamos es añadir border al gráfico.

Los templates pueden combinarse usando `'+'`. De hecho, en los próximos gráficos veremos que como template se especifica la combinación `'plotly_white+borders'`.

In [53]:
import plotly.io as pio
pio.templates["borders"] = go.layout.Template(
    layout = dict(
        xaxis = dict(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black'
        ),
        yaxis = dict(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black'
        )
    )
)

En la siguiente celda modificamos el aspecto de `fig_scatter` con `.update_layout()`. Cuando realizamos estas modificaciones **no** es necesario reasignar el resultado al objeto `fig_scatter`.
Para las modificaciones del layout hay múltiples formas de acceder a los elementos que nos interesan. Aparte de acceder a ellas como diccionarios anidados, se puede accederr a ellas usando `_` para encadenar los sucesivos nombres. Es decir `xaxis={'title':'Number of ratings'}` sería igual que especificar `xaxis_title='Number of ratings'`. 

Para más información al respecto: https://plotly.com/python/reference/index/ 

In [54]:
fig_scatter.update_layout(
    # https://plotly.com/python/templates/ 
    template = 'plotly_white+borders', 
    # https://plotly.com/python/reference/layout/xaxis/
    xaxis = {
        'title':'Number of ratings',
        'range':[0, 4000000]
    }, 
    # https://plotly.com/python/reference/layout/yaxis/
    yaxis = {
        'title':'Average rating',
        'range':[1,5]
    },
    # https://plotly.com/python/reference/layout/#layout-legend
    legend = {
        'title': None,
        'x': 1.02,
        'y': 0.5
    }
)
fig_scatter.show()

Con `customdata` accedemos a las variables que hemos especificado en `hover_data` al crear el gráfico, en el mismo orden en el que las hemos especificado. En nuestro caso dichas variables eran: `['title', 'series_name', 'series_n']`. Para indicar que nos estamos refiriendo a esas variables dentro de `hovertemplate` tenemos que escribir `%`, por ejemplo:`'%{x}'` o `'%{customdata[hoverdata_list_index]}'`

In [55]:
fig_scatter.update_traces(
    # https://plotly.com/python/hover-text-and-formatting/
    hovertemplate = '<b>%{customdata[0]}</b> <br>%{customdata[1]} %{customdata[2]}'
)

En las siguientes celdas calculamos la media de valoración (rating) para cada grupo/color. Posteriormente, añadimos este valor como una línea horizontal que nos permite comparar los tres grupos.

In [56]:
books_df_scatter_means = books_df_scatter.groupby('saga')['average_rating'].mean()
books_df_scatter_means

saga
Saga (continuation)    4.193780
Saga (first book)      4.141667
Standalone             3.955000
Name: average_rating, dtype: float64

In [57]:
for i in range(3) :
    trace_name = fig_scatter.data[i]['legendgroup']
    fig_scatter.add_hline(
        y=books_df_scatter_means[trace_name],
        line_color = fig_scatter.data[i]['marker']['color']
    )

fig_scatter.show()

### Ejemplo 2: Barplot con facetas + añadir trazos

#### Ejemplo 2.1: Una única saga (un único gráfico)

In [58]:
sanderson_sagas_df = pd.read_csv('sanderson_sagas_df.csv')
sanderson_sagas_df.head()

,book_id_long,book_id_short,url,title,series_name,series_url,series_n,isbn,year_first_published,author_name,...,average_rating,main_genre,rating_5_starts,rating_4_starts,rating_3_starts,rating_2_starts,rating_1_start,genres,rating_distribution,hist_rating
0,68428.The_Final_Empire,68428,https://www.goodreads.com/book/show/68428.The_...,The Final Empire,The Mistborn Saga,https://www.goodreads.com/series/40910-the-mis...,1,9780765311788,2006.0,Brandon Sanderson,...,4.46,Fantasy,300783.0,153806.0,39647.0,8207.0,4258.0,NaN,NaN,4.460000
1,68429.The_Well_of_Ascension,68429,https://www.goodreads.com/book/show/68429.The_...,The Well of Ascension,The Mistborn Saga,https://www.goodreads.com/series/40910-the-mis...,2,NaN,2007.0,Brandon Sanderson,...,4.37,Fantasy,181540.0,126733.0,35412.0,5726.0,1837.0,NaN,NaN,4.423154
2,2767793-the-hero-of-ages,2767793,https://www.goodreads.com/book/show/2767793-th...,The Hero of Ages,The Mistborn Saga,https://www.goodreads.com/series/40910-the-mis...,3,NaN,2008.0,Brandon Sanderson,...,4.50,Fantasy,201257.0,91677.0,24761.0,4455.0,1770.0,NaN,NaN,4.444215
3,10803121-the-alloy-of-law,10803121,https://www.goodreads.com/book/show/10803121-t...,The Alloy of Law,The Mistborn Saga,https://www.goodreads.com/series/40910-the-mis...,4,NaN,2011.0,Brandon Sanderson,...,4.21,Fantasy,66578.0,70774.0,23860.0,3191.0,790.0,NaN,NaN,4.415493
4,16065004-shadows-of-self,16065004,https://www.goodreads.com/book/show/16065004-s...,Shadows of Self,The Mistborn Saga,https://www.goodreads.com/series/40910-the-mis...,5,NaN,2015.0,Brandon Sanderson,...,4.29,Fantasy,46918.0,44566.0,12316.0,1461.0,471.0,NaN,NaN,4.406360


In [59]:
fig_stormlight = px.bar(
    sanderson_sagas_df.query('series_name == "The Stormlight Archive"'), 
    x='series_n', y='average_rating', 
    color = 'num_ratings', color_continuous_scale = px.colors.sequential.Teal,
    hover_data = ['title'],
    width = 600, height = 300
    )

fig_stormlight.show()

In [60]:
fig_stormlight.update_layout(
    title = 'The Stormlight Archive',
    coloraxis_colorbar = {'title':'Number<br>of ratings'}, 
    xaxis = {'title':'Series Volume'},
    yaxis = {'title':'Average rating', 'range':[1,5]},
    template = 'plotly_white+borders'
)

In [61]:
fig_stormlight.add_trace(
    go.Scatter(
        x=sanderson_sagas_df.query('series_name == "The Stormlight Archive"')['series_n'],
        y=sanderson_sagas_df.query('series_name == "The Stormlight Archive"')['hist_rating'],
        mode = 'lines+markers', showlegend=False
    )
)

In [62]:
fig_stormlight.update_layout(
    xaxis = {'type':'category'}
)

#### Ejemplo 2.2 Barplot con facetas
Lo más sencillo sería indicar `facet_row` en vez de `facet_col`. De ese modo, no tendríamos que especificar `facet_col_wrap=1` (un único gráfico por fila/la 'anchura' de cada columna es de solo 1 gráfico). PEro cuando se usa `facet_row` el título de cada gráfico aparece a la izquierda y girado 90 grados. Usar `facet_col`+`facet_col_wrap=1` es un truco para conseguir de forma sencilla que el título esté en la parte superior.

In [63]:
fig_sanderson = px.bar(
    sanderson_sagas_df, 
    x='series_n', y='average_rating', 
    facet_col='series_name', facet_col_wrap=1,
    color = 'num_ratings',
    color_continuous_scale = px.colors.sequential.Teal,
    hover_data = ['title', 'num_ratings', 'hist_rating'],
    width = 600, height = 700)

fig_sanderson.show()

In [64]:
fig_sanderson_facet_row = px.bar(
    sanderson_sagas_df, 
    x='series_n', y='average_rating', 
    facet_row='series_name', 
    color = 'num_ratings',
    color_continuous_scale = px.colors.sequential.Teal,
    hover_data = ['title', 'num_ratings', 'hist_rating'],
    width = 600, height = 700)

fig_sanderson_facet_row.show()

Cuando tenemos facet plots, son útiles otros métodos que no hemos usado anteriormente, como: `.for_each_annotation()` (sirve para hacer referencia a todos los títulos y etiquetas de leyenedas) o `.for_each_yaxis()` (para hacer referencia al eje Y de cada subplot). Por defecto, todos los gráficos comparten el mismo rango y por eso podemos cambiarlo con `.update_layout(yaxis_range=[1,5])`. Sin embargo, el título es individual para cada subplot y por eso tenemos que 'borrarlo' con `.for_each_yaxis()`.

In [65]:
fig_sanderson.update_layout(
    coloraxis_colorbar = {'title':'Number<br>of ratings'}, 
    xaxis = {'title':'Series Volume'},
    yaxis = {'range':[1,5]},
    template = 'plotly_white+borders'
)
fig_sanderson.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig_sanderson.for_each_yaxis(lambda yaxis: yaxis.update(title=None))

fig_sanderson.add_annotation(
    text='Average rating', 
    x=-0.11, y=0.5,
    xref="paper", yref="paper",
    textangle=-90, showarrow=False
    )
    
fig_sanderson.update_traces(
    # https://plotly.com/python/hover-text-and-formatting/
    hovertemplate = '<b>%{customdata[0]}</b><br>Num. ratings: %{customdata[1]:.0f} <br>Avg. rating: %{y:.2f}<br>Saga rating: %{customdata[2]:.2f}'
)

`sanderson_sagas` es una lista con los títulos de todas las sagas que aparecen en el gráfico. Usamos `[::-1]` para revertir el orden porque en los facet plots, las filas (row) empiezan a contar desde abajo. Es decir, los números de cada uno de los plots serían, de arriba a abajo: (row=5, col=1), (row=4, col=1), (row=3, col=1), (row=2, col=1), (row=1, col=1).
Al incuir dentro del bucle `fig_sanderson.show()` podemos ver cómo las líneas se van añadiendo de abajo hacia arriba. 

Por otro lado, lo que estamos haciendo en cada vuelta del bucle es añadir un trazo (trace) nuevo a la lista de `data` de la Figura. Cada trazo lo creamos con `go.Scatter()`, la función de `go` para crear trazos del tipo nube de puntos. Hay también funciones go.Line, go.Bar, etc. Para crear líneas se suele usar go.Scatter en vez de go.Line porque es más versátil y, especificando `mode='lines'` o `mode='lines+markers'` el resultado es el mismo.

https://plotly.com/python/facet-plots/#adding-lines-and-rectangles-to-facet-plots

In [66]:
sanderson_sagas = sanderson_sagas_df.series_name.unique()

for i, saga in enumerate(sanderson_sagas[::-1]):
    df_i = sanderson_sagas_df.query(f'series_name == "{saga}"')
    fig_sanderson.add_trace(
        go.Scatter(
            x=df_i['series_n'], y=df_i['hist_rating'], 
            mode='lines+markers', line_color='black',
            showlegend=False, hoverinfo='none'
            ),
        row=i+1, col=1)
    #Descomentar las dos líneas inferiores para ver cómo va cambiando el gráfico en cada iteración
    #print(len(fig_sanderson.data))
    #fig_sanderson.show()

fig_sanderson.show()


También podemos incluir un `rangeslider` para que los usuarios puedan modificar con más facilidad el rango del eje X. Lo malo de esta opción es que 'fija' el eje Y, por lo que ya no se puede aplicar el zoom. Esta opción suele ser útil para gráficos donde el eje X representa fechas. 

In [67]:
# https://plotly.com/python/range-slider/
fig_sanderson.update_layout(
    xaxis = {
        'rangeslider':{
            'autorange':True,
            'visible':True
        }
    }
)